<a href="https://colab.research.google.com/github/Riky2014/Tesi/blob/main/1d_hemo_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
!apt-get install software-properties-common
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install fenics

In [6]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# Data
A0 = 4e-4
q0 = 0
k_r = 2.416e-4
rho = 1.05
alpha = 1

T = 1
x_left = 0
x_right = 1.0

# Discretization parameter
dt = 2e-4
h = 1e-1
num_steps = T / dt
N = int((x_right - x_left) / h)

#  Create a mesh on the interval [0, 1].
mesh = IntervalMesh(N, x_left, x_right)
x = MeshCoordinates(mesh)

# Define the function space
P1 = FiniteElement('P', mesh.ufl_cell(), 1)
element = MixedElement([P1, P1])
V = FunctionSpace(mesh, element)

In [13]:
# Exact solution
L = 1
A_tilde = 4e-4
a_tilde = 4e-5
q_tilde = 0.0
A_exact = Expression('A_tilde + a_tilde * sin(2 * pi / L * x[0]) * cos(2 * pi / T * t)', A_tilde = A_tilde, a_tilde = a_tilde, L = L, T = T, degree = 2, t = 0)
q_exact = Expression('q_tilde - (a_tilde * L / T) * cos(2 * pi / L * x[0]) * sin(2 * pi / T * t)', q_tilde = q_tilde, a_tilde = a_tilde, L = L, T = T,degree = 2, t = 0)

In [14]:
def U(A, q):
  return as_vector([A, q])

def H(A, q):
  return as_tensor([[0, 1], [1190476.19048 * A ** 0.5 - (q / A) ** 2, 2 * q / A]])

def F(A, q):
  return as_vector([q, 793650.79365 * A ** 1.5 - 6.34921 + q ** 2 / A])

def B(A, q):
  return as_vector([0, k_r * q / A])

def S(A, q):
  return as_vector([0, k_r * q / A])

def dS_dU(A, q):
  return as_tensor([[0, 0], [- k_r * q / A ** 2, k_r / A]])

In [ ]:
#  derivative function spoace

D1 = FiniteElement('DG', mesh.ufl_cell(), 0)
element_der = MixedElement([D1, D1])
V_der = FunctionSpace(mesh, element_der)



In [44]:
# Modity in order to pass only the singolor value of each expression at the inlet or at the outlet

def c_alpha(A, q, alpha = 1):
  return (1190476.19048 * A ** 0.5 + (q / A) ** 2 * alpha * (alpha - 1)) ** 0.5

def l1(A, q, alpha = 1):
  return as_vector([c_alpha(A, q, alpha) - alpha * q / A, 1.])

def l2(A, q, alpha = 1):
  return ([- c_alpha(A, q, alpha) - alpha * q / A, 1.])

def CC(A, q,U_der):
  # pass the derivative interpolated on V_der
  return U(A, q) - dt * dot(H(A, q), U(A, q).dx(0)) - dt * B(A, q) + dt * f

In [39]:
uh = Function(V)
uh=interpolate(Constant((1.0,1.0)),V)
Ah, qh = uh.split()

In [47]:
np.dot(l2(Ah(1.), qh(0.)), l2(Ah(1.), qh(0.)) )

1192660.3693823637

In [16]:
def inlet_bc(A, q):
  q_inlet = (
      (project(l2(A, q)[0] * CC(A, q)[0], V.sub(0).collapse()).compute_vertex_values(mesh)
      + project(l2(A, q)[1] * CC(A, q)[1], V.sub(1).collapse()).compute_vertex_values(mesh)
      - project(l2(A, q)[0] * U(A, q)[0], V.sub(0).collapse()).compute_vertex_values(mesh))
      / project(l2(A, q)[1] , V.sub(1).collapse()).compute_vertex_values(mesh)
  )[0]

  return q_inlet




In [17]:
def outlet_matrix(A, q):
  return as_tensor([[1, -1], [c_alpha(A, q, alpha) + alpha * q / A, c_alpha(A, q, alpha) - alpha * q / A]])

def outlet_vector(A, q):
  return as_vector([l1(A, q)[0] * CC(A, q)[0] + l1(A, q)[1] * CC(A, q)[1], l2(A, q)[0] * (U(A, q) - dt * S(A, q))[0] + l2(A, q)[1] * (U(A, q) - dt * S(A, q))[1]])

def outlet_bc(A, q):
  A_out_fun, q_out_fun = project(dot(outlet_matrix(A, q), outlet_vector(A, q)) / (2 * c_alpha(A, q)), V).split()
  A_outlet = A_out_fun.compute_vertex_values(mesh)[-1]
  q_outlet = q_out_fun.compute_vertex_values(mesh)[-1]

  return A_outlet, q_outlet

In [18]:
# Define the source term
f = Expression(('0','5.98399 * pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) * cos(6.28319*t) * cos(6.28319*x[0]) - 0.00025 * cos(6.28319*t) * cos(6.28319*x[0])'), degree = 2, t = 0)
df_dt = Expression(('0', '-1.87993 * sin(6.28319*t) * sin(6.28319*x[0]) * cos(6.28319*t) * cos(6.28319*x[0]) / pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) - 37.5985 * pow((0.1*sin(6.28319 * x[0]) * cos(6.28319*t) + 1), 0.5) * sin(6.28319*t) * cos(6.28319*x[0]) + 0.00157 * sin(6.28319*t) * cos(6.28319*x[0])'), degree = 2, t = 0)

f_n = interpolate(f, V)
df_dt_n = interpolate(df_dt, V)

In [19]:
# # Define the source term
# f = Expression(('0','5.98399 * pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) * cos(6.28319*t) * cos(6.28319*x[0]) - 0.00025 * cos(6.28319*t) * cos(6.28319*x[0]) - 0.0001 * sin(6.28319*t) * cos(6.28319*x[0]) / (4.0e-5 * sin(6.28319*x[0]) * cos(6.28319*t) + 0.0004)'), degree = 2, t = 0)
# df_dt = Expression(('0', '-1.87993 * sin(6.28319*t) * sin(6.28319*x[0]) * cos(6.28319*t) * cos(6.28319*x[0]) / pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) - 37.5985 * pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) * sin(6.28319*t) * cos(6.28319*x[0]) + 0.00157 * sin(6.28319*t) * cos(6.28319*x[0]) - 0.15752 * pow(sin(6.28319*t), 2) * sin(6.28319*x[0])* cos(6.28319*x[0]) / pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 2) - 0.00063 * cos(6.28319*t) * cos(6.28319*x[0]) / (4.0e-5 * sin(6.28319*x[0]) * cos(6.28319*t) + 0.0004)'), degree = 2, t = 0)

# f_n = interpolate(f, V)
# df_dt_n = interpolate(df_dt, V)

In [50]:
# Define initial guess
uh_old = interpolate(Expression(('A_tilde + a_tilde * sin(2 * pi / L * x[0])', 'q0'), degree = 2, A_tilde = A_tilde, a_tilde = a_tilde, L = L, q0 = q0), V)
Ah_old, q_old = uh_old.split()

0.0004

In [ ]:
# Boundary condition
inlet = 'near(x[0], 0)'
outlet = 'near(x[0], 1)'
A_inlet = A0
q_inlet = inlet_bc(A_old, q_old)
A_outlet, q_outlet = outlet_bc(A_old, q_old)
print(f'A inlet = {A_inlet}, q inlet = {q_inlet}')
print(f'A outlet = {A_outlet}, q outlet = {q_outlet}')

# Dirichlet bc
bc_A_inlet = DirichletBC(V.sub(0), A_inlet, inlet)
# I should impose compatibility condition
bc_q_inlet = DirichletBC(V.sub(1), q_inlet, inlet)

# Non reflecting bc
bc_A_outlet = DirichletBC(V.sub(0), A_outlet, outlet)
# I should impose compatibility condition
bc_q_outlet = DirichletBC(V.sub(1), q_outlet, outlet)

bc = [bc_A_inlet, bc_q_inlet, bc_A_outlet, bc_q_outlet]

A inlet = 0.0004, q inlet = 4.081314050279757e-05
A outlet = 0.0004001306839320736, q outlet = 2.0299488021142842e-05


In [ ]:
# Define trial functions and test functions
A, q = TrialFunctions(V)
v, z = TestFunctions(V)

# Define the variational problem
a = inner(A, v) * dx + inner(q, z) * dx

L = (
      A_old * v * dx
    + q_old * z * dx
    + dt * ((F(A_old, q_old) - dt / 2 * dot(H(A_old, q_old), S(A_old, q_old))))[0] * v.dx(0) * dx
    + dt * ((F(A_old, q_old) - dt / 2 * dot(H(A_old, q_old), S(A_old, q_old))))[1] * z.dx(0) * dx
    + dt ** 2 / 2 * (dot(dS_dU(A_old, q_old), F(A_old, q_old).dx(0)))[0] * v * dx
    + dt ** 2 / 2 * (dot(dS_dU(A_old, q_old), F(A_old, q_old).dx(0)))[1] * z * dx
    - dt ** 2 / 2 * (dot(H(A_old, q_old), F(A_old, q_old).dx(0)))[0] * v.dx(0) * dx
    - dt ** 2 / 2 * (dot(H(A_old, q_old), F(A_old, q_old).dx(0)))[1] * z.dx(0) * dx
    - dt * (S(A_old, q_old) - dt / 2 * dot(dS_dU(A_old, q_old), S(A_old, q_old)))[0] * v * dx
    - dt * (S(A_old, q_old) - dt / 2 * dot(dS_dU(A_old, q_old), S(A_old, q_old)))[1] * z * dx
    + dt * f[0] * v * dx
    + dt * f[1] * z * dx
    + dt ** 2 / 2 * (dot(H(A_old, q_old), f_n))[0] * v.dx(0) * dx
    + dt ** 2 / 2 * (dot(H(A_old, q_old), f_n))[1] * z.dx(0) * dx
    + dt ** 2 / 2 * (- dot(dS_dU(A_old, q_old), f_n) + df_dt_n)[0] * v * dx
    + dt ** 2 / 2 * (- dot(dS_dU(A_old, q_old), f_n) + df_dt_n)[1] * z * dx
)

In [ ]:
# Time stepping
u = Function(V)
t = 0
i = 0
for n in range(round(num_steps)):

  i +=1

  # Update time step
  t += dt
  f.t = t
  df_dt.t = t
  A_exact.t = t
  q_exact.t = t

  # Solve the problem
  solve(a == L, u, bc)
  A, q = u.split(deepcopy = True)

  # Compute errors
  if (i % 100 == 0):
    print(f'Interation {i} / {round(num_steps)}')
    A_e = interpolate(A_exact, V.sub(0).collapse())
    q_e = interpolate(q_exact, V.sub(1).collapse())
    error = np.array([errornorm(A, A_e, 'L2'), errornorm(q, q_e, 'L2')])
    #error = np.array([(A.compute_vertex_values() - A_e.compute_vertex_values(mesh)).max(), (q.compute_vertex_values() - q_e.compute_vertex_values(mesh)).max()])
    print(f't = %.4f: error = {error}' % (t))
    print()

  # Update previous solution
  A_old.assign(A)
  q_old.assign(q)

  q_inlet = inlet_bc(A_old, q_old)
  A_outlet, q_outlet = outlet_bc(A_old, q_old)

Interation 100 / 5000
t = 0.0200: error = [9.20946202e-07 2.85563010e-05]

Interation 200 / 5000
t = 0.0400: error = [9.10919406e-07 1.68087861e-05]

Interation 300 / 5000
t = 0.0600: error = [8.91106644e-07 3.42104502e-05]

Interation 400 / 5000
t = 0.0800: error = [8.61853228e-07 7.53405930e-05]

Interation 500 / 5000
t = 0.1000: error = [8.23660486e-07 1.28597948e-04]

Interation 600 / 5000
t = 0.1200: error = [7.77170508e-07 1.91863231e-04]

Interation 700 / 5000
t = 0.1400: error = [7.23151447e-07 2.63864867e-04]

Interation 800 / 5000
t = 0.1600: error = [6.62485213e-07 3.43374555e-04]

Interation 900 / 5000
t = 0.1800: error = [5.96160781e-07 4.29097997e-04]

Interation 1000 / 5000
t = 0.2000: error = [5.25278048e-07 5.19662844e-04]

Interation 1100 / 5000
t = 0.2200: error = [4.51071901e-07 6.13629499e-04]

Interation 1200 / 5000
t = 0.2400: error = [3.74980149e-07 7.09509429e-04]

Interation 1300 / 5000
t = 0.2600: error = [2.98823333e-07 8.05786574e-04]

Interation 1400 / 500